In [83]:
import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import numpy as np
from sklearn.model_selection import train_test_split

In [84]:
torch.__version__

'2.1.2+cu118'

In [85]:
torch.cuda.is_available()

True

In [86]:
class BinaryClassification(nn.Module):
  def __init__(self, embed_size, device):
      super(BinaryClassification, self).__init__()
      # Number of input features is embed_size.
      self.layer_1 = nn.Linear(embed_size, 64).to(device)
      self.layer_2 = nn.Linear(64, 64).to(device)
      self.layer_out = nn.Linear(64, 1).to(device)
      
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(p=0.1)
      self.batchnorm1 = nn.BatchNorm1d(64).to(device)
      self.batchnorm2 = nn.BatchNorm1d(64).to(device)
      self.device = device

  def forward(self, inputs):
    # print("start binary classification")
    # print(inputs.shape)
    # print(inputs)
    x = self.relu(self.layer_1(inputs))
    x = self.batchnorm1(x)
    x = self.relu(self.layer_2(x))
    x = self.batchnorm2(x)
    x = self.dropout(x)
    x = self.layer_out(x)
    #if math.isnan (x[0][0]):
    #  print(src)

    return x

In [87]:
# class Classifier(nn.Module):
#   def __init__(self, d_model, seq_len, nhead, dim_feedforward, nlayers, device, dropout = 0.5):
#     super(Classifier, self).__init__()
#     self.d_model = d_model
#     self.seq_len = seq_len
#     self.nhead = nhead
#     self.dim_feedforward = dim_feedforward
#     self.nlayers = nlayers
#     self.device = device
#     #self.pos_encoder = PositionalEncoding(d_model, dropout)
#     self.position_embedding = nn.Embedding(seq_len, d_model)
#     encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
#     self.encoder = TransformerEncoder(encoder_layer, nlayers)
#     self.binary_classifier = BinaryClassification(seq_len*d_model, device)


#   def forward(self, src: Tensor) -> Tensor:
#     # print("Classifier forwrd")
#     """
#     Args:
#         src: Tensor, shape [seq_len, batch_size]
#         src_mask: Tensor, shape [seq_len, seq_len]
#     Returns:
#         output Tensor of shape [seq_len, batch_size, ntoken]
#     """
#     N, seq_length, embed_size = src.shape
#     positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

#     # print(f"src before positional embeddings: {src.shape}")
#     src_ = src + self.position_embedding(positions)
#     # print(f"src after positional embeddings: {src_.shape}")
#     # print(f"src after positional embeddings: {src.shape}")
#     # print(src_)
#     # print("before encoder")
#     output = self.encoder(src_)
#     # print(f"encoder output shape: {output.shape}")
#     # print(output[0])
#     # print(output)
#     # print("after encoder")
#     output = self.binary_classifier(torch.reshape(output, (N, seq_length*embed_size))) 
#     # output_tx_f = torch.reshape(output, (N, seq_length*embed_size))
#     # output = self.binary_classifier(torch.cat((output_tx_f), dim=1)) ##
#     return output

In [88]:
class Classifier(nn.Module):
  def __init__(self, d_model, seq_len, nhead, dim_feedforward, nlayers, device, dropout = 0.5):
    super(Classifier, self).__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.nhead = nhead
    self.dim_feedforward = dim_feedforward
    self.nlayers = nlayers
    self.device = device
    
    self.position_embedding = nn.Embedding(seq_len, d_model)
    encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
    self.encoder = TransformerEncoder(encoder_layer, nlayers)
    self.binary_classifier = BinaryClassification(seq_len*d_model, device)

  def forward(self, src: Tensor) -> Tensor:

    """
    Args:
        src: Tensor, shape [seq_len, batch_size]
        src_mask: Tensor, shape [seq_len, seq_len]
    Returns:
        output Tensor of shape [seq_len, batch_size, ntoken]
    """
    N, seq_length, embed_size = src.shape
    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
    src_ = src + self.position_embedding(positions)
    output1 = self.encoder(src_)
    # print(output1.shape)
    # print(output1)
    output = self.binary_classifier(torch.reshape(output1, (N, seq_length*embed_size))) 

    return output, output1

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#hyperparameters
dim_feedforward=16
#dim_feedforward=32
nlayers_tx=8

In [90]:
model = Classifier(d_model=11, seq_len=108, nhead=11, dim_feedforward=8, nlayers=8, device=device)
model.to(device)

c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Classifier(
  (position_embedding): Embedding(108, 11)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=11, out_features=11, bias=True)
        )
        (linear1): Linear(in_features=11, out_features=8, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=8, out_features=11, bias=True)
        (norm1): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (binary_classifier): BinaryClassification(
    (layer_1): Linear(in_features=1188, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=1, bias=True)
    

In [91]:
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [92]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    # print("y_pred",torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [93]:
x = np.load("data\other\data.npy")
y = np.load("data\other\labels.npy")

In [94]:
print(x.shape)
print(y.shape)

(6184, 108, 11)
(6184,)


In [95]:
unique_labels, label_counts = np.unique(y, return_counts=True)
current_label_ratio = dict(zip(unique_labels, label_counts / len(y)))
print("Current Label Ratio:", current_label_ratio)

Current Label Ratio: {0: 0.9589262613195343, 1: 0.04107373868046572}


In [96]:
print(x[:5])

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   2.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[6.25000000e-01 6.25000000e-01 1.08000000e+20 ... 8.00000000e+00
   2.00000000e+00 1.00000000e+01]
  [1.00000000e+00 1.00000000e+00 1.00000000e+19 ... 9.00000000e+00
   1.00000000e+00 1.00000000e+01]
  [5.71428571e-01 5.71428571e-01 1.90000000e+19 ... 7.00000000e+00
   1.00000000e+00 1.00000000e+01]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.00000000e+00
   0.00000

In [97]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming you have loaded your data into a variable called data_array
# data_array should be of shape (301, 108, 11)

# Reshape the array to (301*108, 11) for normalization
reshaped_data = x.reshape((-1, 11))

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the reshaped data and transform it
normalized_data = scaler.fit_transform(reshaped_data)

# Reshape the normalized data back to the original shape
normalized_data_array = normalized_data.reshape(x.shape)

# Now, normalized_data_array contains the normalized data with shape (301, 108, 11)
print(normalized_data_array[:5])

[[[-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
    4.89447156e+00 -2.26839474e-01]
  [-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
    2.23285053e+00 -2.26839474e-01]
  [-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
    2.23285053e+00 -2.26839474e-01]
  ...
  [-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
   -4.28770492e-01 -2.26839474e-01]
  [-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
   -4.28770492e-01 -2.26839474e-01]
  [-5.76306301e-02 -7.21936347e-02 -3.25123457e-03 ... -1.74238883e-01
   -4.28770492e-01 -2.26839474e-01]]

 [[ 1.80722002e+01  1.37837402e+01  6.96871979e-03 ...  8.11181005e+00
    4.89447156e+00  4.49345481e+00]
  [ 2.89500988e+01  2.20973005e+01 -2.30494250e-03 ...  9.14756616e+00
    2.23285053e+00  4.49345481e+00]
  [ 1.65182147e+01  1.25960887e+01 -1.45327963e-03 ...  7.07605393e+00
    2.23285053e+00  4.49345481e+00]
  ...
  [-5.76306301e-02 -7.2

In [98]:
X, X_test, Y, y_test = train_test_split(normalized_data_array, y, test_size=0.1, random_state=42, stratify=y) #test split

In [99]:
import numpy as np


# Print current label ratio
unique_labels, label_counts = np.unique(Y, return_counts=True)
current_label_ratio = dict(zip(unique_labels, label_counts / len(Y)))
print("Current Label Ratio:", current_label_ratio)

# Balance the dataset to achieve a 1:5 ratio without random oversampling
desired_ratio = {0: 1, 1: 5}  # Adjust this ratio as needed

balanced_x = []
balanced_y = []

for label, ratio in desired_ratio.items():
    label_indices = np.where(Y == label)[0]
    label_data = X[label_indices]
    
    # Replicate data to achieve the desired ratio
    replicated_data = np.tile(label_data, (ratio, 1, 1))
    
    balanced_x.append(replicated_data)
    balanced_y.extend([label] * len(replicated_data))

balanced_x = np.concatenate(balanced_x)
balanced_y = np.array(balanced_y)

# Verify the new label ratio
unique_balanced_labels, balanced_label_counts = np.unique(balanced_y, return_counts=True)
new_label_ratio = dict(zip(unique_balanced_labels, balanced_label_counts / len(balanced_y)))
print("Balanced Label Ratio:", new_label_ratio)

# Now, 'balanced_x' and 'balanced_y' contain the balanced dataset


Current Label Ratio: {0: 0.9588499550763702, 1: 0.04115004492362983}
Balanced Label Ratio: {0: 0.8233297330658849, 1: 0.1766702669341151}


In [100]:

# Assuming you have split the data using the code provided in the previous response

# Check class distribution in original 'y'
unique_classes_y, counts_y = np.unique(y, return_counts=True)
print("Original class distribution:")
print(dict(zip(unique_classes_y, counts_y)))

# Check class distribution in 'y_train'
unique_classes_train, counts_train = np.unique(Y, return_counts=True)
print("\nClass distribution in y_train:")
print(dict(zip(unique_classes_train, counts_train)))

# Check class distribution in 'y_train_balance'
unique_classes_train, counts_train = np.unique(balanced_y, return_counts=True)
print("\nClass distribution in y_train_balance:")
print(dict(zip(unique_classes_train, counts_train)))

# Check class distribution in 'y_test'
unique_classes_test, counts_test = np.unique(y_test, return_counts=True)
print("\nClass distribution in y_test:")
print(dict(zip(unique_classes_test, counts_test)))


Original class distribution:
{0: 5930, 1: 254}

Class distribution in y_train:
{0: 5336, 1: 229}

Class distribution in y_train_balance:
{0: 5336, 1: 1145}

Class distribution in y_test:
{0: 594, 1: 25}


In [101]:
np.save("X_test.npy",X_test)
np.save("y_test.npy",y_test)

In [102]:
#load test
X_test = np.load("X_test.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

X=balanced_x
Y=balanced_y

In [103]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y) #validation split

In [104]:
np.save("X_train.npy",X_train)
np.save("y_train.npy",y_train)
np.save("X_val.npy",X_val)
np.save("y_val.npy",y_val)

In [105]:
#load train val
X_train = np.load("X_train.npy", allow_pickle=True)
y_train = np.load("y_train.npy", allow_pickle=True)
X_val = np.load("X_val.npy", allow_pickle=True)
y_val = np.load("y_val.npy", allow_pickle=True)

In [106]:
print(X_train.shape)

(5832, 108, 11)


In [107]:
X_train = X_train[:19656]

In [108]:
print(X_val.shape)

(649, 108, 11)


In [109]:
X_val = X_val[:2184]

In [110]:
print(X_test.shape)

(619, 108, 11)


In [111]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [112]:
print(np.sum(y_train)/y_train.shape[0])
print(np.sum(y_val)/y_val.shape[0])
print(np.sum(y_test)/y_test.shape[0])

0.17661179698216736
0.17719568567026195
0.04038772213247173


In [113]:
from torch.utils.data import Dataset, DataLoader

In [114]:
## train data
from sklearn import preprocessing
class TrainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(X_train, y_train)

## val data
class ValData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


val_data = ValData(X_val, y_val)

## test data
class TestData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


test_data = TestData(X_test, y_test)

In [115]:
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001

In [116]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [117]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [118]:
print(len(train_loader))
print(train_loader)

183


Hyperparameter tuning

In [119]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# import seaborn as sn
import pandas as pd

# Define the hyperparameter values to test
hyperparameters = {
    'epochs': [ 20],
    'batch_size': [ 16,32,64],
    'learning_rate': [0.0005,0.001, 0.003,0.004,0.005,]
}

# Generate all combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Iterate over each combination
for params in param_combinations:
    # Unpack the parameters
    epochs, batch_size, learning_rate = params

    # Create a new model and optimizer with the current hyperparameters
    model = Classifier(d_model=11, seq_len=108, nhead=11, dim_feedforward=8, nlayers=8, device=device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Training loop
    for e in range(1, epochs + 1):
        # ... (your existing training code)
        epoch_loss = 0
        epoch_acc = 0
        model.train()
        for X_batch, y_batch in train_loader:
            #print("w.requires_grad:",X_batch.requires_grad)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred,result_before_bin_classifier = model(X_batch.float())
            # print(f"y_pred = {y_pred}")
            #y_pred = (y_pred>0.5).float()
            #print(f"y_pred = {y_pred}")
            #print(f"y_batch.unsqueeze(1) = {y_batch.unsqueeze(1)}")

            loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
            acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())


            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        val_loss = 0
        val_accuracy = 0
        model.eval()
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred,result_before_bin_classifier = model(X_batch.float())
            loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
            acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())
            val_loss += loss.item()
            val_accuracy += acc.item()

        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f} | Val_Loss: {val_loss/len(test_loader):.5f} | Val_Acc: {val_accuracy/len(test_loader):.3f}')

    y_pred = []
    y_true = []
    sigmoid = nn.Sigmoid()

    for X_batch, y_batch in test_loader:
            #print("w.requires_grad:",X_batch.requires_grad)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # print(X_batch.shape)
            # print(y_batch.shape)
            results,result_before_bin_classifier = model(X_batch.float())
            y_batch_pred = torch.round(sigmoid(results))
            y_pred.extend(y_batch_pred.cpu().detach().numpy())
            y_true.extend(y_batch.cpu().detach().numpy())

     # Print the results for each hyperparameter combination
    print(f'Hyperparameters: Epochs={epochs}, Batch Size={batch_size}, Learning Rate={learning_rate}')
    print(f'Final Training Loss: {epoch_loss/len(train_loader):.5f}, Final Training Acc: {epoch_acc/len(train_loader):.3f}')
    print(f'Final Validation Loss: {val_loss/len(test_loader):.5f}, Final Validation Acc: {val_accuracy/len(test_loader):.3f}')
    print('\n')

    count_true = 0
    for i in range(len(y_pred)):
        if y_true[i]==y_pred[i]:
            count_true+=1
    acc = count_true/len(y_pred)
    print("accuracy: ",acc)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print(tn, fp, fn, tp)

    # Calculate Recall
    recall = tp / (tp + fn)
    # Calculate Precision
    precision = tp / (tp + fp)
    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall)

    print("Recall: {:.4f}".format(recall))
    print("Precision: {:.4f}".format(precision))
    print("F1 Score: {:.4f}".format(f1_score))

            
   


c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 001: | Loss: 0.59921 | Acc: 70.421 | Val_Loss: 0.35498 | Val_Acc: 90.307
Epoch 002: | Loss: 0.48977 | Acc: 82.011 | Val_Loss: 0.28658 | Val_Acc: 90.468
Epoch 003: | Loss: 0.42906 | Acc: 84.060 | Val_Loss: 0.26270 | Val_Acc: 92.730
Epoch 004: | Loss: 0.40497 | Acc: 84.628 | Val_Loss: 0.21919 | Val_Acc: 92.892
Epoch 005: | Loss: 0.39609 | Acc: 85.301 | Val_Loss: 0.32580 | Val_Acc: 92.084
Epoch 006: | Loss: 0.39135 | Acc: 85.361 | Val_Loss: 0.30001 | Val_Acc: 92.730
Epoch 007: | Loss: 0.38590 | Acc: 85.585 | Val_Loss: 0.20903 | Val_Acc: 93.861
Epoch 008: | Loss: 0.38435 | Acc: 85.678 | Val_Loss: 0.20584 | Val_Acc: 94.992
Epoch 009: | Loss: 0.37726 | Acc: 86.213 | Val_Loss: 0.20553 | Val_Acc: 94.669
Epoch 010: | Loss: 0.37382 | Acc: 86.404 | Val_Loss: 0.20910 | Val_Acc: 94.023
Epoch 011: | Loss: 0.37246 | Acc: 86.410 | Val_Loss: 0.19361 | Val_Acc: 95.638
Epoch 012: | Loss: 0.36862 | Acc: 86.585 | Val_Loss: 0.22700 | Val_Acc: 94.830
Epoch 013: | Loss: 0.36657 | Acc: 86.770 | Val_Loss:

c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 001: | Loss: 0.54705 | Acc: 77.667 | Val_Loss: 0.70092 | Val_Acc: 91.438
Epoch 002: | Loss: 0.42795 | Acc: 83.989 | Val_Loss: 0.18349 | Val_Acc: 95.477
Epoch 003: | Loss: 0.40588 | Acc: 84.421 | Val_Loss: 0.32799 | Val_Acc: 95.961
Epoch 004: | Loss: 0.38942 | Acc: 85.579 | Val_Loss: 0.19646 | Val_Acc: 96.446
Epoch 005: | Loss: 0.38250 | Acc: 86.022 | Val_Loss: 0.32875 | Val_Acc: 94.830
Epoch 006: | Loss: 0.38239 | Acc: 85.973 | Val_Loss: 0.24689 | Val_Acc: 95.800
Epoch 007: | Loss: 0.37885 | Acc: 86.366 | Val_Loss: 0.25396 | Val_Acc: 94.507
Epoch 008: | Loss: 0.36895 | Acc: 86.803 | Val_Loss: 0.20250 | Val_Acc: 95.477
Epoch 009: | Loss: 0.37172 | Acc: 86.590 | Val_Loss: 0.24669 | Val_Acc: 96.446
Epoch 010: | Loss: 0.38190 | Acc: 86.033 | Val_Loss: 0.29377 | Val_Acc: 94.023
Epoch 011: | Loss: 0.36929 | Acc: 86.404 | Val_Loss: 0.24870 | Val_Acc: 94.992
Epoch 012: | Loss: 0.37807 | Acc: 86.120 | Val_Loss: 0.21066 | Val_Acc: 95.153
Epoch 013: | Loss: 0.37297 | Acc: 86.464 | Val_Loss:

c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 001: | Loss: 0.47940 | Acc: 79.803 | Val_Loss: 0.23462 | Val_Acc: 88.368
Epoch 002: | Loss: 0.40579 | Acc: 84.027 | Val_Loss: 3.39833 | Val_Acc: 83.037
Epoch 003: | Loss: 0.40328 | Acc: 84.978 | Val_Loss: 0.42349 | Val_Acc: 89.338
Epoch 004: | Loss: 0.39967 | Acc: 84.443 | Val_Loss: 0.23404 | Val_Acc: 88.691
Epoch 005: | Loss: 0.40032 | Acc: 84.251 | Val_Loss: 0.25360 | Val_Acc: 93.053
Epoch 006: | Loss: 0.38328 | Acc: 86.202 | Val_Loss: 0.29330 | Val_Acc: 92.407
Epoch 007: | Loss: 0.37534 | Acc: 86.464 | Val_Loss: 0.28373 | Val_Acc: 92.892
Epoch 008: | Loss: 0.37219 | Acc: 86.574 | Val_Loss: 0.25960 | Val_Acc: 93.700
Epoch 009: | Loss: 0.37684 | Acc: 86.459 | Val_Loss: 0.26059 | Val_Acc: 92.730
Epoch 010: | Loss: 0.37253 | Acc: 86.683 | Val_Loss: 0.30242 | Val_Acc: 92.407
Epoch 011: | Loss: 0.39720 | Acc: 84.508 | Val_Loss: 0.36036 | Val_Acc: 88.853
Epoch 012: | Loss: 0.39621 | Acc: 84.393 | Val_Loss: 0.25989 | Val_Acc: 90.468
Epoch 013: | Loss: 0.39256 | Acc: 84.913 | Val_Loss:

c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 001: | Loss: 0.45982 | Acc: 81.689 | Val_Loss: 0.28418 | Val_Acc: 90.307
Epoch 002: | Loss: 0.40862 | Acc: 83.694 | Val_Loss: 0.29437 | Val_Acc: 89.015
Epoch 003: | Loss: 0.40619 | Acc: 84.301 | Val_Loss: 0.30462 | Val_Acc: 90.468
Epoch 004: | Loss: 0.38832 | Acc: 85.568 | Val_Loss: 0.34997 | Val_Acc: 89.338
Epoch 005: | Loss: 0.38977 | Acc: 85.186 | Val_Loss: 0.22512 | Val_Acc: 95.153
Epoch 006: | Loss: 0.38372 | Acc: 85.623 | Val_Loss: 0.26227 | Val_Acc: 91.761
Epoch 007: | Loss: 0.37710 | Acc: 86.104 | Val_Loss: 0.26183 | Val_Acc: 93.700
Epoch 008: | Loss: 0.38378 | Acc: 85.962 | Val_Loss: 0.66592 | Val_Acc: 91.761
Epoch 009: | Loss: 0.44675 | Acc: 83.240 | Val_Loss: 0.17774 | Val_Acc: 95.961
Epoch 010: | Loss: 0.46988 | Acc: 82.311 | Val_Loss: 0.30571 | Val_Acc: 95.961
Epoch 011: | Loss: 0.47380 | Acc: 82.383 | Val_Loss: 0.28342 | Val_Acc: 95.961
Epoch 012: | Loss: 0.47053 | Acc: 82.377 | Val_Loss: 0.22133 | Val_Acc: 95.961
Epoch 013: | Loss: 0.46918 | Acc: 82.388 | Val_Loss:

C:\Users\sandu\AppData\Local\Temp\ipykernel_9452\3152788074.py:101: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 001: | Loss: 0.46874 | Acc: 79.781 | Val_Loss: 0.35586 | Val_Acc: 86.914
Epoch 002: | Loss: 0.42713 | Acc: 82.634 | Val_Loss: 0.23763 | Val_Acc: 88.530
Epoch 003: | Loss: 0.41845 | Acc: 82.825 | Val_Loss: 0.45728 | Val_Acc: 94.184
Epoch 004: | Loss: 0.40749 | Acc: 84.230 | Val_Loss: 0.22027 | Val_Acc: 94.184
Epoch 005: | Loss: 0.42826 | Acc: 84.098 | Val_Loss: 0.15444 | Val_Acc: 95.961
Epoch 006: | Loss: 0.40921 | Acc: 84.984 | Val_Loss: 0.38827 | Val_Acc: 95.477
Epoch 007: | Loss: 0.42296 | Acc: 82.689 | Val_Loss: 0.25908 | Val_Acc: 88.368
Epoch 008: | Loss: 0.42370 | Acc: 83.066 | Val_Loss: 0.20401 | Val_Acc: 95.961
Epoch 009: | Loss: 0.42405 | Acc: 83.230 | Val_Loss: 0.19681 | Val_Acc: 89.822
Epoch 010: | Loss: 0.42018 | Acc: 83.213 | Val_Loss: 0.21749 | Val_Acc: 90.307
Epoch 011: | Loss: 0.41911 | Acc: 83.158 | Val_Loss: 0.18758 | Val_Acc: 89.822
Epoch 012: | Loss: 0.40433 | Acc: 85.443 | Val_Loss: 0.30678 | Val_Acc: 91.761
Epoch 013: | Loss: 0.39688 | Acc: 86.552 | Val_Loss:

In [75]:

# for e in range(1, EPOCHS+1):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.train()
#     for X_batch, y_batch in train_loader:
#         #print("w.requires_grad:",X_batch.requires_grad)
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         optimizer.zero_grad()

#         y_pred,result_before_bin_classifier = model(X_batch.float())
#         # print(f"y_pred = {y_pred}")
#         #y_pred = (y_pred>0.5).float()
#         #print(f"y_pred = {y_pred}")
#         #print(f"y_batch.unsqueeze(1) = {y_batch.unsqueeze(1)}")

#         loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
#         acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())


#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()
#         epoch_acc += acc.item()
#     val_loss = 0
#     val_accuracy = 0
#     model.eval()
#     for X_batch, y_batch in test_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         y_pred,result_before_bin_classifier = model(X_batch.float())
#         loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
#         acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())
#         val_loss += loss.item()
#         val_accuracy += acc.item()

#     # Check for early stopping
#         # if val_accuracy > best_validation_accuracy:
#         #     best_validation_accuracy = val_accuracy
#         #     no_improvement_counter = 0
#         #     # Save the trained best fakebert if needed
#         #     torch.save(fakebert.state_dict(), '/content/drive/Shareddrives/test/FYP/fake-news/fakebert-twitterus.pth')
#         # else:
#         #     no_improvement_counter += 1

#     print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f} | Val_Loss: {val_loss/len(test_loader):.5f} | Val_Acc: {val_accuracy/len(test_loader):.3f}')
#     #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.56920 | Acc: 75.399 | Val_Loss: 0.46614 | Val_Acc: 88.530
Epoch 002: | Loss: 0.43354 | Acc: 83.077 | Val_Loss: 0.81745 | Val_Acc: 94.346
Epoch 003: | Loss: 0.40778 | Acc: 84.197 | Val_Loss: 0.41433 | Val_Acc: 87.237
Epoch 004: | Loss: 0.40460 | Acc: 83.929 | Val_Loss: 0.28373 | Val_Acc: 90.630
Epoch 005: | Loss: 0.40387 | Acc: 84.344 | Val_Loss: 0.32396 | Val_Acc: 89.015
Epoch 006: | Loss: 0.39395 | Acc: 84.896 | Val_Loss: 0.30900 | Val_Acc: 90.953
Epoch 007: | Loss: 0.38999 | Acc: 85.000 | Val_Loss: 0.23921 | Val_Acc: 93.700
Epoch 008: | Loss: 0.38847 | Acc: 85.295 | Val_Loss: 0.30056 | Val_Acc: 92.892
Epoch 009: | Loss: 0.38425 | Acc: 85.760 | Val_Loss: 0.25420 | Val_Acc: 94.023
Epoch 010: | Loss: 0.37647 | Acc: 86.574 | Val_Loss: 0.49931 | Val_Acc: 91.438
Epoch 011: | Loss: 0.37870 | Acc: 86.393 | Val_Loss: 0.23069 | Val_Acc: 94.023
Epoch 012: | Loss: 0.37374 | Acc: 86.710 | Val_Loss: 0.22753 | Val_Acc: 94.507
Epoch 013: | Loss: 0.37386 | Acc: 86.503 | Val_Loss:

In [76]:
#PATH = './DeFi_Supervised_v1.pth'
#PATH = './DeFi_Ablation_v2.pth'
PATH = './PonziShield_tr_v5_gpu.pth'
torch.save(model, PATH)




In [77]:
#PATH = './DeFi_Ablation_v1.pth'
PATH = './PonziShield_tr_v5_gpu.pth'
# PATH = './PonziShield_tr_v3.pth'
loaded_model = torch.load(PATH)
loaded_model.eval()

Classifier(
  (position_embedding): Embedding(108, 11)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=11, out_features=11, bias=True)
        )
        (linear1): Linear(in_features=11, out_features=8, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=8, out_features=11, bias=True)
        (norm1): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (binary_classifier): BinaryClassification(
    (layer_1): Linear(in_features=1188, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=1, bias=True)
    

In [78]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# import seaborn as sn
import pandas as pd

In [79]:
y_pred = []
y_true = []
sigmoid = nn.Sigmoid()

for X_batch, y_batch in test_loader:
        #print("w.requires_grad:",X_batch.requires_grad)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # print(X_batch.shape)
        # print(y_batch.shape)
        results,result_before_bin_classifier = loaded_model(X_batch.float())
        y_batch_pred = torch.round(sigmoid(results))
        y_pred.extend(y_batch_pred.cpu().detach().numpy())
        y_true.extend(y_batch.cpu().detach().numpy())

In [80]:
unique_classes_train, counts_train = np.unique(y_test, return_counts=True)
print("\nClass distribution in y_test:")
print(dict(zip(unique_classes_train, counts_train)))

count_true = 0
for i in range(len(y_pred)):
  if y_true[i]==y_pred[i]:
    count_true+=1
acc = count_true/len(y_pred)
print("accuracy: ",acc)


Class distribution in y_test:
{0: 594, 1: 25}
accuracy:  0.9499192245557351


In [81]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print("tn-",tn ,"fp-", fp,"fn-", fn,"tp-", tp)

tn- 580 fp- 14 fn- 17 tp- 8


In [82]:
# Calculate Recall
recall = tp / (tp + fn)

# Calculate Precision
precision = tp / (tp + fp)

# Calculate F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))

Recall: 0.3200
Precision: 0.3636
F1 Score: 0.3404


In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1), index = ['0','1'],
                     columns = ['0','1'])
plt.figure(figsize = (12,7))
# sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

<Figure size 1200x700 with 0 Axes>

For one DAPP

**Two token**

In [ ]:
#Anubis
ankh_x1 = np.load("ankh_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0071]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.2284]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9375]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#full

#for i in range(50,108):
#  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[7.5255e-07]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Sushi
sushi_x1 = np.load("sushi_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0580]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9919]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9957]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Full

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[1.]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

**APE**

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving ape_full_x1.npy to ape_full_x1.npy


In [ ]:
#Ape
x1 = np.load("ape_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0006]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9763]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9997]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Full

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[8.2050e-08]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)